In [11]:
import numpy as np
import pandas as pd
import gzip
import json
import os

In [2]:
path = r'../../../Results/'
file = r'Concatenated_MFile.gz'
pathFile = path + file

In [3]:
dataList = []

with gzip.open(pathFile, 'r') as dataFile:
    for line in dataFile:
        lineData = json.loads(line.decode('utf-8'))
        dataList.append(lineData)
data = pd.DataFrame(dataList)

In [4]:
len(data)

192665

In [7]:
data["origin"].nunique()

1928

In [31]:
path = r'../../../../src/data/Restaurant/Restaurant_minimum3/geo_preprocessed'

In [32]:
len(os.listdir(path))

1093

## Conclusion our Concatenated_MFile has too less entries to begin with

- Option 1: Use additional Data?


- Option 2: **use a less strict appraoch in the first place**


In [7]:
data.longitude = data.longitude.round(5)
data.latitude = data.latitude.round(5)

In [25]:
cluster = data.groupby(['E.164 format', 'latitude', 'longitude']).size().reset_index(name='counts')
cluster = cluster.sort_values('counts', ascending=False)
pd.set_option('max_colwidth', 400)

In [81]:
cluster

,E.164 format,latitude,longitude,counts
28585,+17136954939,29.79909,-95.36764,2053
66568,+443458121007,51.55645,0.70062,1701
63719,+442035985088,51.42955,-0.12394,1506
58697,+421905383904,48.14383,17.85810,515
69152,+48570600559,49.95232,18.59726,416
...,...,...,...,...
49121,+18887964652,33.69740,-78.87371,1
49122,+18887964652,33.72066,-78.87742,1
21083,+16019567079,32.36586,-90.14700,1
49124,+18887964652,33.72519,-78.84001,1


In [80]:
cluster['counts'] > 200

28585     True
66568     True
63719     True
58697     True
69152     True
         ...  
49121    False
49122    False
21083    False
49124    False
40881    False
Name: counts, Length: 81763, dtype: bool

In [82]:
phoneNumbers = cluster['E.164 format'].loc[cluster['counts'] > 200]
data = data[~data['E.164 format'].isin(phoneNumbers)]

In [83]:
cluster_clean = data.groupby(['E.164 format', 'latitude', 'longitude']).size().reset_index(name='counts')
cluster_clean = cluster_clean.sort_values('counts', ascending=False)
cluster_clean

,E.164 format,latitude,longitude,counts
5630,+12813921166,29.70628,-95.74338,195
56640,+34932554084,41.41580,2.13866,192
63275,+441934521155,51.36261,-2.91280,192
61994,+441623646000,53.15599,-1.18753,187
12022,+14046364800,33.88618,-84.26237,180
...,...,...,...,...
49100,+18887867384,42.00718,-87.82309,1
49101,+18887867384,42.96273,-85.63750,1
49103,+18887919099,40.73744,-74.04313,1
49105,+18887964652,33.47594,-79.11422,1


In [84]:
len(data)

180823

In [ ]:
file = 'LB_ClusteringResult.gz'
data.to_json(path + file, compression="gzip", orient='records', lines=True)